## Introduction and Context
This notebook extends our previous **Darcy Flow FNO** prototype by illustrating an **offline active learning** (AL) workflow. We assume we have a **trained PDE surrogate**—in particular, a dropout-enabled model (e.g., `FNOWithDropout`). The essence of **offline AL** is to identify “where the model is most uncertain” and select those PDE inputs for additional simulation, partial retraining, or deeper inspection. We’ll use **MC-Dropout** to measure the variance of multiple stochastic forward passes, guiding us to the PDE fields that are likely the hardest or “least understood” by the current model.

## Table of Contents
1. [Introduction and Context](#introduction-and--context)  
   - Explains how **offline active learning** builds upon the **trained PDE surrogate** from the first notebook.  
   - Summarizes the concept of using **MC-Dropout** to identify uncertain PDE samples.

2. [AL_01 Setup & Configuration](#al_01-setup-and-configuration)  
   - Loads `config.yaml` or Hydra configuration for AL (checkpoint paths, dropout rate, num of MC passes, etc.).  
   - Imports necessary libraries (`torch`, `Darcy2D`, `MCDropoutEstimator`, etc.).

3. [AL_02 Model Loading](#al_02-model-loading)  
   - Constructs the **dropout-enabled** operator (e.g., `FNOWithDropout`) with matching hyperparams from the prior notebook.  
   - Loads the trained checkpoint (`.pt`), sets device (`cpu` or `cuda`).

4. [AL_03 Candidate Data Generation](#al_03-candidate-data-generation)  
   - Illustrates generating a batch of PDE inputs (e.g., via `Darcy2D`) for the AL procedure.  
   - Optionally visualizes or logs the shape/stats of these candidate fields.

5. [AL_04 MC-Dropout Uncertainty Estimation](#al_04-mc-dropout-uncertainty-estimation)  
   - Explains how **MCDropoutEstimator** runs multiple stochastic forward passes.  
   - Computes mean and variance across passes for each candidate PDE input, storing them in `mean_pred` and `var_pred`.

6. [AL_05 Selecting Top-K Uncertain Samples](#al_05-selecting-top-k-uncertain-samples)  
   - Aggregates uncertainty (e.g., averaging `var_pred` across spatial dims).  
   - Picks the top-K uncertain PDE samples, logs or prints them for clarity.

7. [AL_06 Save & (Optionally) Retrain](#al_06-save--optionally-retrain)  
   - Creates a `.pt` dictionary with the chosen PDE inputs (and predictions) for potential partial retraining or HPC PDE solves.  
   - Mentions how we’d add these new PDE fields back into the main dataset.

8. [AL_07 Optional Repeated AL Loop](#al_07-optional-repeated-al-loop)  
   - Summarizes how we could iterate the procedure: generate → pick top-K → add to dataset → retrain → repeat.  
   - More relevant for a “live” or multi-step AL scenario.

10. [Conclusion & Next Steps](#conclusion--next-steps)  
    - Recaps the **offline AL** approach for Darcy Flow.  
    - Hints at advanced usage: HPC parallel generation, repeated AL loops, or integration with the first notebook’s AutoML pipeline.

> **Reference**: For data generation, model building, or hyperparameter tuning steps, see our **[Main Notebook](darcy_autoML.ipynb)**. This AL notebook specifically focuses on uncertainty-driven sample selection once a dropout-based PDE surrogate is trained.

### AL_01 Setup and Configuration
In this section, we **load our AL-specific configuration** (e.g., checkpoint paths, dropout rate, number of Monte Carlo passes, and the top-K selection size). We also import essential dependencies such as PyTorch, the Darcy PDE generator, and our custom `MCDropoutEstimator`. This step ensures all parameters are aligned with the environment used in the previous notebook.

### AL_02 Model Loading
Here, we **construct the dropout-based PDE surrogate** (e.g., `FNOWithDropout`) with the same hyperparameters used in training, then **load the trained checkpoint**. We switch the model to `.eval()` mode but keep dropout layers “active” for the MC passes. This stage verifies the model’s integrity and device setup before proceeding to uncertainty estimation.

### AL_03 Candidate Data Generation
Next, we demonstrate how to **generate a batch of PDE inputs** (e.g., using the `Darcy2D` data loader) for the AL procedure. We pull a single batch (or more, if needed) of permeability fields, which serve as our “candidates.” Optionally, we may visualize or summarize these input tensors (e.g., shape checks) to confirm correctness.

### AL_04 MC-Dropout Uncertainty Estimation
This section details the **core of the AL process**—**MC-Dropout**. Using `MCDropoutEstimator`, we run multiple forward passes on each candidate PDE input, capturing **mean** and **variance** predictions for each sample. This reveals how “confident” or “uncertain” the surrogate is about each PDE field. The resulting arrays `mean_pred` and `var_pred` contain our per-sample signals for subsequent top-K selection.

### AL_05 Selecting Top-K Uncertain Samples
We now **aggregate** the voxel-wise variances (e.g., average them spatially) to get a single scalar “uncertainty” per candidate sample. We then **pick the top-K** PDE fields where the model is least certain—those presumably requiring additional solver runs or retraining data. This section prints or logs the top uncertain items, helping you see which PDE inputs are outliers or otherwise uncharted territory for the current surrogate.

### AL_06 Save & (Optionally) Retrain
Once we have the top-K uncertain samples, we can **save them** into a `.pt` dictionary (including their input fields, mean predictions, and variance maps). In a subsequent step—possibly in the first notebook’s pipeline or HPC environment—these PDE samples could be **added** to the training set for partial retraining. This ensures future surrogates become more robust in precisely the scenarios that cause uncertainty today.

### AL_07 Optional Repeated AL Loop
Here, we highlight how to **cycle** the offline AL approach for multiple rounds—**generate → pick top-K → retrain → repeat**. Although this notebook focuses on a single demonstration pass, more advanced workflows might iterate until the PDE surrogate’s uncertainty (and validation error) converge to acceptable levels. This approach can be integrated with HPC or a live PDE solver to refine the surrogate in real-time.


### Conclusion & Next Steps
Finally, we **recap** how offline AL, powered by **MC-Dropout**, helps prioritize PDE samples for further simulation or partial retraining. With minimal overhead, you can systematically hone the surrogate where it struggles most. Going forward, you might incorporate HPC parallel data generation, or weave AL into the **[Main Notebook](darcy_autoML.ipynb)** pipeline for a cohesive PDE–AI ecosystem. This ensures an ever-improving operator model, driven by uncertainty-driven sample additions.